In [1]:
import numpy as np

In [2]:
class Alphabet(dict):
    def __init__(self, start_id=1):
        self.fid = start_id
    
    def add(self, item):
        idx = self.get(item, None)
        if idx is None:
            idx = self.fid
            self[item] = idx
            self.fid += 1
        return idx
    
    def dump(self, fname):
        with open(fname, 'w') as out:
            for k in sorted(self.keys()):
                out.write("{}\t{}\n".format(k, self[k]))

In [3]:
def load_embed(fname):
    f = open(fname)
    cnt, vocab_size, embed_dim = 0, 0, 0
    embed_dict = {}
    print 'Load embedding file start!'
    for line in f:
        cnt += 1
        if cnt % 10000 == 0:
            print cnt
        terms = line.strip().split(' ')
        if len(terms) == 2:
            vocab_size = int(terms[0])
            embed_dim = int(terms[1])
        if len(terms) == embed_dim + 1:
            embed_dict[terms[0]] = np.array([float(ii) for ii in terms[1:]])
    print 'Load embedding file finish!'
    return embed_dict, vocab_size, embed_dim

In [4]:
def parse_embed(embed_file):
    alphabet, embed_mat = Alphabet(), []
    embed_dict, vocab_size, embed_dim = load_embed(embed_file)
    unknown_word_idx = 0
    embed_mat.append(np.random.uniform(-0.25, 0.25, embed_dim))
    for word in embed_dict:
        alphabet.add(word)
        embed_mat.append(embed_dict[word])
    dummy_word_idx = alphabet.fid
    embed_mat.append(np.zeros(embed_dim))
    return alphabet, embed_mat, unknown_word_idx, dummy_word_idx

In [5]:
char_embed_file = '../ieee_zhihu_cup/char_embedding.txt'
#word_embed_file = '../ieee_zhihu_cup/word_embedding.txt'
char_alphabet, char_embed_mat, unknown_char_idx, dummy_char_idx = parse_embed(char_embed_file)
#word_alphabet, word_embed_mat, unknown_word_idx, dummy_word_idx = parse_embed(word_embed_file)

Load embedding file start!
10000
Load embedding file finish!


In [6]:
def load_question(fname, split=False, rate=None):
    f = open(fname)
    cnt, idx, title_char, title_word, desc_char, desc_word = 0, [], [], [], [], []
    print 'Load question file start!'
    for line in f:
        cnt += 1
        if cnt % 10000 == 0:
            print cnt
        terms = line.strip().split('\t')
        idx.append(terms[0])
        if len(terms) == 5:
            title_char.append(terms[1])
            title_word.append(terms[2])
            desc_char.append(terms[3])
            desc_word.append(terms[4])
        elif len(terms) == 4:
            title_char.append(terms[1])
            title_word.append(terms[2])
            desc_char.append(terms[3])
            desc_word.append('')
        elif len(terms) == 3:
            title_char.append(terms[1])
            title_word.append(terms[2])
            desc_char.append('')
            desc_word.append('')
        elif len(terms) == 1:
            title_char.append('')
            title_word.append('')
            desc_char.append('')
            desc_word.append('')
    print 'Load question file finish!'
    if split:
        ids = np.arange(cnt)
        np.random.seed(1024)
        np.random.shuffle(ids)
        train_id = ids[:int(cnt*rate)]
        val_id = ids[int(cnt*rate):]
        print "Finished"
        return [idx[i] for i in train_id], [title_char[i] for i in train_id], \
                [title_word[i] for i in train_id], [desc_char[i] for i in train_id], \
                [desc_word[i] for i in train_id], \
                [idx[i] for i in val_id], [title_char[i] for i in val_id], \
                [title_word[i] for i in val_id], [desc_char[i] for i in val_id], \
                [desc_word[i] for i in val_id]
    print "Finished"
    return idx, title_char, title_word, desc_char, desc_word

In [7]:
def convert2indices(data, alphabet, unknown_word_idx, dummy_word_idx, max_length):
    data_idx = []
    for item in data:
        item_arr = [ii for ii in item.split(',') if ii != '']
        ex = np.ones(max_length) * dummy_word_idx
        for i, word in enumerate(item_arr):
            if i >= max_length:
                break
            idx = alphabet.get(word, unknown_word_idx)
            ex[i] = idx
        data_idx.append(ex)
    data_idx = np.array(data_idx).astype('int32')
    return data_idx

In [8]:
question_train_file = '../ieee_zhihu_cup/question_train_set.txt'
question_test_file = '../ieee_zhihu_cup/question_eval_set.txt'
train_idx, train_title_char, train_title_word, train_desc_char, train_desc_word, \
val_idx, val_title_char, val_title_word, val_desc_char, val_desc_word \
            = load_question(question_train_file, split=True, rate=0.9)
train_idx_all, train_title_char_all, train_title_word_all, train_desc_char_all, \
                train_desc_word_all = load_question(question_train_file)
test_idx, test_title_char, test_title_word, test_desc_char, test_desc_word \
                           = load_question(question_test_file)

Load question file start!
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
13

In [9]:
title_char_max_length = 85#22
#title_word_max_length = 50#30#13
desc_char_max_length = 300#117
#desc_word_max_length = 150#120#58

In [10]:
train_title_char_indices = convert2indices(train_title_char, char_alphabet, \
                                           unknown_char_idx, dummy_char_idx, \
                                           title_char_max_length)
#train_title_word_indices = convert2indices(train_title_word, word_alphabet, \
#                                           unknown_word_idx, dummy_word_idx, \
#                                           title_word_max_length)
train_desc_char_indices = convert2indices(train_desc_char, char_alphabet, \
                                           unknown_char_idx, dummy_char_idx, \
                                           desc_char_max_length)
#train_desc_word_indices = convert2indices(train_desc_word, word_alphabet, \
#                                           unknown_word_idx, dummy_word_idx, \
#                                           desc_word_max_length)

In [11]:
val_title_char_indices = convert2indices(val_title_char, char_alphabet, \
                                           unknown_char_idx, dummy_char_idx, \
                                           title_char_max_length)
#val_title_word_indices = convert2indices(val_title_word, word_alphabet, \
#                                           unknown_word_idx, dummy_word_idx, \
#                                           title_word_max_length)
val_desc_char_indices = convert2indices(val_desc_char, char_alphabet, \
                                           unknown_char_idx, dummy_char_idx, \
                                           desc_char_max_length)
#val_desc_word_indices = convert2indices(val_desc_word, word_alphabet, \
#                                           unknown_word_idx, dummy_word_idx, \
#                                           desc_word_max_length)

In [12]:
basedir = './embed'
np.save('{}/char_embed_mat.npy'.format(basedir), char_embed_mat)
del char_embed_mat
#np.save('{}/word_embed_mat.npy'.format(basedir), word_embed_mat)
#del word_embed_mat
print 'Save embedding finished!'
basedir = './train'
#np.save('{}/train_idx.npy'.format(basedir), train_idx)
#del train_idx
np.save('{}/train_title_char_indices_2.npy'.format(basedir), train_title_char_indices)
del train_title_char_indices
#np.save('{}/train_title_word_indices_2.npy'.format(basedir), train_title_word_indices)
#del train_title_word_indices
np.save('{}/train_desc_char_indices_2.npy'.format(basedir), train_desc_char_indices)
del train_desc_char_indices
#np.save('{}/train_desc_word_indices_2.npy'.format(basedir), train_desc_word_indices)
#del train_desc_word_indices
print 'Save train data finished!'
basedir = './val'
#np.save('{}/val_idx.npy'.format(basedir), val_idx)
#del val_idx
np.save('{}/val_title_char_indices_2.npy'.format(basedir), val_title_char_indices)
del val_title_char_indices
#np.save('{}/val_title_word_indices_2.npy'.format(basedir), val_title_word_indices)
#del val_title_word_indices
np.save('{}/val_desc_char_indices_2.npy'.format(basedir), val_desc_char_indices)
del val_desc_char_indices
#np.save('{}/val_desc_word_indices_2.npy'.format(basedir), val_desc_word_indices)
#del val_desc_word_indices
print 'Save val data finished!'

Save embedding finished!
Save train data finished!
Save val data finished!


In [13]:
train_title_char_indices_all = convert2indices(train_title_char_all, char_alphabet, \
                                           unknown_char_idx, dummy_char_idx, \
                                           title_char_max_length)
#train_title_word_indices_all = convert2indices(train_title_word_all, word_alphabet, \
#                                           unknown_word_idx, dummy_word_idx, \
#                                           title_word_max_length)
train_desc_char_indices_all = convert2indices(train_desc_char_all, char_alphabet, \
                                           unknown_char_idx, dummy_char_idx, \
                                           desc_char_max_length)
#train_desc_word_indices_all = convert2indices(train_desc_word_all, word_alphabet, \
#                                           unknown_word_idx, dummy_word_idx, \
#                                           desc_word_max_length)

In [14]:
test_title_char_indices = convert2indices(test_title_char, char_alphabet, \
                                           unknown_char_idx, dummy_char_idx, \
                                           title_char_max_length)
#test_title_word_indices = convert2indices(test_title_word, word_alphabet, \
#                                           unknown_word_idx, dummy_word_idx, \
#                                           title_word_max_length)
test_desc_char_indices = convert2indices(test_desc_char, char_alphabet, \
                                           unknown_char_idx, dummy_char_idx, \
                                           desc_char_max_length)
#test_desc_word_indices = convert2indices(test_desc_word, word_alphabet, \
#                                           unknown_word_idx, dummy_word_idx, \
#                                           desc_word_max_length)

In [15]:
basedir = './train_all'
#np.save('{}/train_idx_all.npy'.format(basedir), train_idx_all)
np.save('{}/train_title_char_indices_all_2.npy'.format(basedir), train_title_char_indices_all)
#np.save('{}/train_title_word_indices_all_2.npy'.format(basedir), train_title_word_indices_all)
np.save('{}/train_desc_char_indices_all_2.npy'.format(basedir), train_desc_char_indices_all)
#np.save('{}/train_desc_word_indices_all_2.npy'.format(basedir), train_desc_word_indices_all)
print 'Save train all data finished!'
basedir = './test'
#np.save('{}/test_idx.npy'.format(basedir), test_idx)
np.save('{}/test_title_char_indices_2.npy'.format(basedir), test_title_char_indices)
#np.save('{}/test_title_word_indices_2.npy'.format(basedir), test_title_word_indices)
np.save('{}/test_desc_char_indices_2.npy'.format(basedir), test_desc_char_indices)
#np.save('{}/test_desc_word_indices_2.npy'.format(basedir), test_desc_word_indices)
print 'Save test data finished!'

Save train all data finished!
Save test data finished!
